In [16]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [18]:
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

In [19]:
import pandas as pd
import numpy as np

#Подготовка Данных

In [285]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_test.csv')

In [286]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25062 entries, 0 to 25061
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              25062 non-null  object 
 1   Category          25062 non-null  object 
 2   Brand             25061 non-null  object 
 3   Seller            25031 non-null  object 
 4   Color             127 non-null    object 
 5   Comments          25062 non-null  float64
 6   Final price       25062 non-null  float64
 7   Max price         25062 non-null  float64
 8   Min price         25062 non-null  float64
 9   Average price     19264 non-null  float64
 10  Sales             25062 non-null  float64
 11  Days in stock     25062 non-null  float64
 12  Days with sales   25062 non-null  int64  
 13  Rating            25062 non-null  object 
 14  Basic Sale        25062 non-null  float64
 15  Basic Sale Price  25062 non-null  float64
 16  Base price        25062 non-null  float6

Добавим среднюю цену среди категории

In [287]:
avg_price_per_category = []
for cat in np.unique(train_data['Category'].values):
    # print(cat)
    avg_price_per_category.append(train_data[train_data['Category'] == cat]['Average price'].mean())
    cat_mean_price = train_data[train_data['Category'] == cat]['Average price'].mean()
    train_data.loc[train_data['Category'] == cat, 'Avg_price_per_category'] = cat_mean_price
# print(avg_price_per_category)
# train_data.head()

avg_price_per_category = []
for cat in np.unique(test_data['Category'].values):
    # print(cat)
    avg_price_per_category.append(test_data[test_data['Category'] == cat]['Average price'].mean())
    cat_mean_price = test_data[test_data['Category'] == cat]['Average price'].mean()
    test_data.loc[test_data['Category'] == cat, 'Avg_price_per_category'] = cat_mean_price

Добавим фичу, если средняя цена продажи не выше чем средняя цена по сегменту - 1, иначе 0




In [288]:
train_data['avg_price_not_more_than_evg_cat'] = np.where((train_data['Avg_price_per_category'] >= train_data['Average price']), 1, 0)

test_data['avg_price_not_more_than_evg_cat'] = np.where((test_data['Avg_price_per_category'] >= test_data['Average price']), 1, 0)

# train_data.head()

In [289]:
# print(f"mean: {train_data['Comments'][:len(train_data['Comments'].values)].mean()}")
print(f"mean: {train_data.sort_values(by='Comments').loc[:len(train_data.sort_values(by='Comments')['Comments'].values)-1, 'Comments'].mean()}")

# train_data.sort_values(by='Comments').loc[:len(train_data.sort_values(by='Comments')['Comments'].values)-1, 'Comments'].value_counts()

mean: 4.590824942430126


In [290]:
train_data['comments_more_than_avg'] = (train_data['Comments'] > 5).astype('int')
# pd.crosstab(train_data['comments_more_than_avg'], train_data['Sales'], margins=True)
test_data['comments_more_than_avg'] = (test_data['Comments'] > 5).astype('int')

Удаляем элементы с продажами более 1000 (меньше 1% всех записей)

In [269]:
# train_data = train_data.sort_values(by='Sales', ascending=False)
# train_data.head()
# for i in range(150):
#     train_data.drop([0], axis=0)

train_data = train_data[train_data['Sales'] < 1000.0]

# train_data['sales_more_than_avg'] = np.where((train_data['Sales'] < 1000.0), 1, 0)

# train_data['sales_more_than_avg'].value_counts()

In [259]:
train_data.head()

,Name,Category,Brand,Seller,Color,Comments,Final price,Max price,Min price,Average price,Sales,Days in stock,Days with sales,Rating,Basic Sale,Basic Sale Price,Base price,full_category,sales_more_than_avg
18146,Весы кухонные электронные / для питания / прод...,/Техника для кухни/Кухонные весы,Eco Ness,Козлов Алексей Николаевич ИП,NaN,3921.0,378.0,378.0,378.0,378.000000,12156.0,31.0,31,"5,0000",55.0,504.0,1120.0,Бытовая техника/Техника для кухни/Кухонные весы,1
17252,Пульсоксиметр,/Красота и здоровье/Медицинские приборы/Пульсо...,FINGER O2XYGEN,Ганин Сергей Владимирович,NaN,1799.0,965.0,965.0,965.0,965.000000,10693.0,30.0,31,"4,0000",80.0,965.0,4828.0,Бытовая техника/Красота и здоровье/Медицинские...,1
18690,Контактный гриль Tefal Inicio GC241D38,/Техника для кухни/Приготовление блюд/Грили эл...,Tefal,ВАЙЛДБЕРРИЗ ООО,NaN,1167.0,3995.0,3995.0,3595.0,3730.237886,9101.0,28.0,30,"5,0000",50.0,3995.0,7990.0,Бытовая техника/Техника для кухни/Приготовлени...,1
12537,Электрическая зубная щётка ORAL-B Vitality D10...,/Красота и здоровье/Зубные щетки и ирригаторы/...,Oral-B,ВАЙЛДБЕРРИЗ ООО,NaN,759.0,2153.0,2153.0,1722.0,1796.360077,7304.0,16.0,27,"5,0000",0.0,2153.0,2153.0,Бытовая техника/Красота и здоровье/Зубные щетк...,1
19743,Машинка против катышек MW-3101,/Техника для дома/Уход за вещами,MAXWELL,ГОЛДЕР-ЭЛЕКТРОНИКС ООО,NaN,1750.0,389.0,528.0,340.0,452.976307,6542.0,20.0,24,"5,0000",19.0,486.0,601.0,Бытовая техника/Техника для дома/Уход за вещами,1


In [291]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25062 entries, 0 to 25061
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Name                             25062 non-null  object 
 1   Category                         25062 non-null  object 
 2   Brand                            25061 non-null  object 
 3   Seller                           25031 non-null  object 
 4   Color                            127 non-null    object 
 5   Comments                         25062 non-null  float64
 6   Final price                      25062 non-null  float64
 7   Max price                        25062 non-null  float64
 8   Min price                        25062 non-null  float64
 9   Average price                    19264 non-null  float64
 10  Sales                            25062 non-null  float64
 11  Days in stock                    25062 non-null  float64
 12  Days with sales   

In [293]:
def deleteId(ds):
    return ds.drop(columns='Id')

def deleteNaN(train_ds, test_ds, critval):
    fullsize = train_ds.shape[0]
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    for feature in new_train.columns:
        nulls = new_train[feature].isnull().sum()
        percent = nulls / fullsize
        # если доля пустых значений превышает critval - столбец не информативен,
        # можно его выбросить
        if (percent > critval):
            new_train = new_train.drop(columns=feature)
            new_test = new_test.drop(columns=feature)
    return new_train, new_test

def convertToNumeric(train_ds, test_ds):
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    LE = LabelEncoder()
    for feature in new_train.columns:
        if (new_train[feature].dtype == 'object'):
            new_train[feature] = LE.fit_transform(new_train[feature])
            new_test[feature] = LE.fit_transform(new_test[feature])
    return new_train, new_test

train = train_data.drop_duplicates()

# train = train.drop(columns='Id')
train = train.drop(columns='Color')
# train = train.drop(columns='Base price')
# train = train.drop(columns='Basic Sale Price')

print(train_data.values.shape)

test = test_data.drop(columns='Id')
test = test.drop(columns='Color')
# test = test.drop(columns='Base price')
# test = test.drop(columns='Basic Sale Price')

print(test.values.shape)

train, test = deleteNaN(train, test, critval=0.8)
train, test = convertToNumeric(train, test)

train_y = train['Sales'].values
train_X = train.drop(columns='Sales').values
val_test = test.values

train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=7)  # random_state=98987)

(25062, 21)
(10741, 19)


In [294]:
print(train_x[0])

[1.16710000e+04 5.50000000e+01 2.44500000e+03 2.07900000e+03
 3.00000000e+00 1.58300000e+03 1.59900000e+03 1.58300000e+03
 1.59038462e+03 6.00000000e+00 5.00000000e+00 5.00000000e+00
 3.70000000e+01 1.58300000e+03 2.51300000e+03 5.50000000e+01
 2.65953012e+03 1.00000000e+00 0.00000000e+00]


#Избавимся от NaN значений


In [295]:
strategies = ['mean', 'median', 'most_frequent']
imputer = SimpleImputer(strategy=strategies[2])
trainX = imputer.fit_transform(train_x)
testX = imputer.fit_transform(test_x)

In [296]:
# pca = PCA(n_components = 16)
# trainX = pca.fit_transform(trainX, train_y)
# testX = pca.transform(testX)

scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [297]:
trainX.shape

(22086, 19)

In [298]:
val_test.shape

(10741, 19)

#Метрика **SMAPE**

In [299]:
def smape(A, F):
    with np.errstate(divide='ignore', invalid='ignore'):
        tmp = 2 * np.abs(F-A) / (np.abs(A) + np.abs(F))
    tmp[np.isnan(tmp)] = 0
    return np.sum(tmp) / len(tmp) * 100

In [300]:
def check_error(preds, gt):
    # print('SMAPE Error:', smape(np.log(preds+1), np.log(gt+1)))
    print('SMAPE Error:', smape(preds, gt))
    # print('RMSE Error:', mean_squared_error(np.round(np.log(np.abs(preds)+1)), np.log(gt+1), squared=False))

#Random Forest

In [20]:
parameters = {
    'criterion':(['absolute_error']), 
    'max_depth':  [300],
    'n_estimators': [70],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': range(10, 91, 10)}

model = RandomForestRegressor()
rf_gs_model = GridSearchCV(model, parameters)
rf_gs_model.fit(trainX[:300], train_y[:300])
print(f"Best parameters: {rf_gs_model.best_params_}")

Best parameters: {'criterion': 'absolute_error', 'max_depth': 300, 'min_samples_leaf': 3, 'min_samples_split': 50, 'n_estimators': 70}


In [ ]:
last_rf_best_params = {
    'criterion': 'absolute_error', 
    'max_depth': 700,  # 300
    'n_estimators': 70, # 70
    'min_samples_leaf': 5, # 3
    'min_samples_split': 50 # 50
}

# random_forest = RandomForestRegressor(**rf_gs_model.best_params_)
random_forest = RandomForestRegressor(**last_rf_best_params) 
random_forest.fit(trainX, train_y)
check_error(random_forest.predict(testX), test_y)

In [16]:
check_error(random_forest.predict(testX), test_y)
print(random_forest.predict(testX))

SMAPE Error: 21.929525404389334
[524.48571429  17.48571429  76.87142857 ...   0.          37.82857143
  21.38571429]


#XGBoost

In [112]:
parameters = {
    "learning_rate": np.linspace(0.2, 0.075, 5),
    "max_depth": range(3, 11, 2),
    "min_child_weight": range(5, 16, 2),
    "n_estimators": range(10, 51, 10),
    "subsample": np.linspace(0.5, 0.9, 5),
    "gamma": np.linspace(0.5, 1, 5),
    'colsample_bytree': [i / 10.0 for i in range(6, 11)]
}

model = XGBRegressor()
xgb_gs_model = GridSearchCV(model, parameters)
xgb_gs_model.fit(trainX[:300], train_y[:300])
print(f"Best parameters: {xgb_gs_model.best_params_}")

Выходные данные были обрезаны до нескольких последних строк (5000).
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:16] WARNING: /workspace/src/objectiv

In [284]:
last_xgb_best_params = {
    "learning_rate": 0.013,
    "max_depth": 15,
    "min_child_weight": 13,
    "n_estimators": 200, 
    "subsample": 0.9,
    "booster": 'gbtree',
    "reg_lambda": 1.,
    "gamma": 0.5,
    'colsample_bytree': 1.
}
xgb_reg = XGBRegressor(**last_xgb_best_params) 
# xgb_reg = XGBRegressor(**xgb_gs_model.best_params_)
xgb_reg.fit(trainX, train_y)
check_error(np.round(np.abs(xgb_reg.predict(testX))), test_y)

[11:06:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SMAPE Error: 25.19938383337632


#CatBoost

In [136]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_test.csv')

In [140]:
train = train_data.drop_duplicates()
# train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.8)

train_y = train['Sales'].values
train = train.drop(columns='Sales')

categorical = []
features = train.columns
for i in range(len(features)):
    if (train[features[i]].dtype == 'object'):
        categorical.append(i) 

train_X = train.values
val_test = test.values

train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=7)

imp = SimpleImputer(strategy='most_frequent')
trainX = imp.fit_transform(train_x)
testX = imp.fit_transform(test_x)

In [148]:
cat_model = CatBoostRegressor(
                            # loss_function= SPAMEObjective(), 
                              loss_function='MAPE', 
                            #   bagging_temperature=1000,
                            #   random_strength=15,
                              cat_features=categorical,
                              learning_rate=.01,
                              depth=3,
                              iterations=2500,
                            #   bootstrap_type='Bernoulli',
                              grow_policy='SymmetricTree',
                            #   l2_leaf_reg=.25
                              )
cat_model.fit(trainX, train_y)
check_error(cat_model.predict(testX), test_y)

0:	learn: 0.7662702	total: 57ms	remaining: 2m 22s
1:	learn: 0.7622076	total: 112ms	remaining: 2m 19s
2:	learn: 0.7574890	total: 155ms	remaining: 2m 9s
3:	learn: 0.7535265	total: 192ms	remaining: 1m 59s
4:	learn: 0.7496036	total: 246ms	remaining: 2m 2s
5:	learn: 0.7457200	total: 280ms	remaining: 1m 56s
6:	learn: 0.7422386	total: 367ms	remaining: 2m 10s
7:	learn: 0.7387818	total: 419ms	remaining: 2m 10s
8:	learn: 0.7347462	total: 449ms	remaining: 2m 4s
9:	learn: 0.7309085	total: 481ms	remaining: 1m 59s
10:	learn: 0.7265020	total: 549ms	remaining: 2m 4s
11:	learn: 0.7229391	total: 593ms	remaining: 2m 2s
12:	learn: 0.7181838	total: 655ms	remaining: 2m 5s
13:	learn: 0.7146283	total: 727ms	remaining: 2m 9s
14:	learn: 0.7111080	total: 800ms	remaining: 2m 12s
15:	learn: 0.7076073	total: 858ms	remaining: 2m 13s
16:	learn: 0.7041590	total: 910ms	remaining: 2m 12s
17:	learn: 0.7000092	total: 941ms	remaining: 2m 9s
18:	learn: 0.6964323	total: 969ms	remaining: 2m 6s
19:	learn: 0.6931114	total: 1s	r

#Соревы

In [276]:
strategies = ['mean', 'median', 'most_frequent']
imputer = SimpleImputer(strategy=strategies[2])
val_test_x = imputer.fit_transform(val_test)

scaler = StandardScaler()
val_test_x = scaler.fit_transform(val_test_x)

In [277]:
submission_path = '/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_sample_solution.csv'
submission = pd.read_csv(submission_path)

In [278]:
predictions = np.round(random_forest.predict(val_test_x))
submission['Expected'] = predictions
submission
submission.to_csv('submission_rf.csv', index=False)

In [223]:
predictions = np.round(xgb_reg.predict(val_test_x))
submission['Expected'] = predictions
submission
submission.to_csv('submission_xgboost.csv', index=False)

ValueError: ignored